# Lab 4: Tools and MCP - Workshop
 
Welcome to **Section 4**: Tools and MCP!
 
In this section, we'll explore how to empower language model agents with external tools and a Multi-Component Pipeline (MCP) for more advanced reasoning and automation. We'll build on the backend logic from `main.py` to demonstrate how agents can interact with APIs, databases, and structured workflows.

Here's what we'll cover:
- **4.1 Planning + Reasoning (TODO list):**  
  Learn how agents can break down complex tasks into actionable steps, plan their approach, and delegate subtasks—just like a human would when facing a big project.
 - **4.2 APIs:**  
   Discover how to connect your agent to external APIs, enabling it to fetch real-time data, perform actions, or integrate with other services.
 - **4.3 Querying Database (SQL):**  
   See how agents can interact with databases using SQL, allowing them to retrieve, update, or analyze structured information as part of their workflow.
 - **4.4 MCP Structure:**  
   Understand the architecture of a Multi-Component Pipeline (MCP), which orchestrates multiple tools and reasoning steps to solve more sophisticated problems.
 
 Throughout this lab, we'll build simple, practical examples to illustrate each concept and help you apply these techniques in your own projects!


In [ ]:
# Setup
import os
from dotenv import load_dotenv
from pathlib import Path

# Load environment  
project_root = Path.cwd()
if project_root.name != 'agent-mastery-course':
    project_root = project_root.parent
load_dotenv(project_root / 'backend' / '.env')

# LangChain imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage  
from langchain_openai import ChatOpenAI
from openinference.instrumentation.mcp import MCPInstrumentor

print("✅ Setup complete! Ready to build tools like in main.py")


In [ ]:
from arize.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.instrumentation.langchain import LangChainInstrumentor

# configure the Phoenix tracer
tracer_provider = register(
    space_id=os.getenv("ARIZE_SPACE_ID"),
    api_key=os.getenv("ARIZE_API_KEY"),
    project_name="lab4-tools-and-mcp",
)
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)
LangChainInstrumentor().instrument(tracer_provider=tracer_provider, include_chains=True, include_agents=True, include_tools=True)
MCPInstrumentor().instrument(tracer_provider=tracer_provider)

## 4.1 Planning + Reasoning (TODO list)

Create agents that can break down complex tasks into manageable steps.


In [ ]:
# 4.1 - Orchestrator agent with LangGraph (plan + delegate)
from typing import List, Dict, Any
from typing_extensions import TypedDict, Annotated
import operator

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import SystemMessage, BaseMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Simple worker tools
@tool
def flights_worker(task: str) -> str:
    """Handle flight and visa related tasks."""
    return f"Flights Agent: will handle '{task}'"

@tool
def hotels_worker(task: str) -> str:
    """Handle hotel and accommodation tasks."""
    return f"Hotels Agent: will handle '{task}'"

@tool
def itinerary_worker(task: str) -> str:
    """Handle itinerary planning tasks."""
    return f"Itinerary Agent: will handle '{task}'"

@tool
def logistics_worker(task: str) -> str:
    """Handle local transport and logistics tasks."""
    return f"Logistics Agent: will handle '{task}'"

class PlanState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    destination: str
    assignments: Annotated[List[str], operator.add]

def _pick_worker(task: str) -> str:
    low = task.lower()
    if "flight" in low or "visa" in low:
        return flights_worker.name  # type: ignore[attr-defined]
    if "hotel" in low or "accommodation" in low:
        return hotels_worker.name  # type: ignore[attr-defined]
    if "itinerary" in low or "plan" in low:
        return itinerary_worker.name  # type: ignore[attr-defined]
    return logistics_worker.name  # type: ignore[attr-defined]

def orchestrator_node(state: PlanState) -> PlanState:
    destination = state["destination"]
    tools = [flights_worker, hotels_worker, itinerary_worker, logistics_worker]
    agent = llm.bind_tools(tools)

    prompt = (
        "You are an orchestrator. Create 5 concise tasks for planning a trip to {destination}.\n"
        "For each task, immediately call exactly one tool among flights_worker, hotels_worker, itinerary_worker, logistics_worker"
        " with the task string. Output only tool calls."
    )

    res = agent.invoke([SystemMessage(content=prompt.format(destination=destination))])

    assignments: List[str] = []
    if getattr(res, "tool_calls", None):
        tool_node = ToolNode(tools)
        tr = tool_node.invoke({"messages": [res]})
        for m in tr["messages"]:
            txt = getattr(m, "content", "")
            if txt:
                assignments.append(txt)
    else:
        # Fallback: simple deterministic plan + assignment if the model doesn't emit tool calls
        fallback_tasks = [
            "Book flights",
            "Reserve accommodation",
            "Create daily itinerary",
            "Research local transport options",
            "Check visa requirements",
        ]
        for t in fallback_tasks: 
            w = _pick_worker(t)
            if w == flights_worker.name:  # type: ignore[attr-defined]
                assignments.append(flights_worker.invoke({"task": t}))
            elif w == hotels_worker.name:  # type: ignore[attr-defined]
                assignments.append(hotels_worker.invoke({"task": t}))
            elif w == itinerary_worker.name:  # type: ignore[attr-defined]
                assignments.append(itinerary_worker.invoke({"task": t}))
            else:
                assignments.append(logistics_worker.invoke({"task": t}))

    return {"messages": [SystemMessage(content="orchestration_complete")], "assignments": assignments}

# Build and run the mini-graph
G = StateGraph(PlanState)
G.add_node("orchestrator", orchestrator_node)
G.add_edge(START, "orchestrator")
G.add_edge("orchestrator", END)
planner_graph = G.compile()

result = planner_graph.invoke({"messages": [], "destination": "Tokyo", "assignments": []})
print("🧠 Orchestrated plan + delegation (LangGraph):")
for line in result.get("assignments", []):
    print("  - " + str(line))



# Try it yourself

- Run the previous cell to generate a TODO list with the LLM and see a simple delegation simulation.
- Change the destination (e.g., "Paris", "Bangkok") to observe different TODOs.
- Optional: extend `delegate_tasks` to include due dates or priorities.



## 4.2 APIs

Learn how to integrate external APIs with agents (simulated based on main.py patterns).


In [ ]:
# API tools (budget_basics and attraction_prices)
import time

@tool 
def get_flight_prices(destination: str, duration: str) -> str:
    """Get flight prices for a destination (simulated API call)."""
    # Simulate API latency and error handling
    time.sleep(0.3)
    
    # Mock API data (like main.py's deterministic approach)
    prices = {
        "Tokyo": "$800-1200",
        "Paris": "$600-900", 
        "Bangkok": "$700-1000"
    }
    
    city = destination.split(",")[0].strip()
    price = prices.get(city, "$500-800")
    
    return f"✈️ Flight prices to {destination} ({duration}): {price} round-trip"

@tool
def get_hotel_rates(destination: str, duration: str) -> str:
    """Get hotel rates for a destination (simulated API call)."""
    # Simulate API call with error handling
    try:
        time.sleep(0.2)
        
        # Mock hotel data (like main.py's budget approach)
        rates = {
            "Tokyo": "$100-300/night",
            "Paris": "$80-250/night",
            "Bangkok": "$50-150/night" 
        }
        
        city = destination.split(",")[0].strip()
        rate = rates.get(city, "$75-200/night")
        
        return f"🏨 Hotel rates in {destination} ({duration}): {rate}"
        
    except Exception as e:
        return f"API Error: Could not fetch hotel rates - {str(e)}"

# Test API tools
print("🌐 Testing API integration:")
print(get_flight_prices.invoke({"destination": "Tokyo", "duration": "7 days"}))
print(get_hotel_rates.invoke({"destination": "Tokyo", "duration": "7 days"}))


## 4.3 Querying Database (SQL)

Build agents that can generate and execute SQL queries safely.


In [ ]:
# Simple database querying
import sqlite3

# Create a simple in-memory database
def create_test_db():
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()
    
    # Create sample travel data table
    cursor.execute('''
        CREATE TABLE destinations (
            id INTEGER PRIMARY KEY,
            city TEXT,
            country TEXT, 
            avg_temp INTEGER,
            best_season TEXT
        )
    ''')
    
    # Insert sample data
    destinations = [
        (1, 'Tokyo', 'Japan', 22, 'Spring'),
        (2, 'Paris', 'France', 18, 'Summer'),
        (3, 'Bangkok', 'Thailand', 30, 'Winter')
    ]
    
    cursor.executemany(
        'INSERT INTO destinations VALUES (?, ?, ?, ?, ?)', 
        destinations
    )
    
    conn.commit()
    return conn

# Initialize database
db = create_test_db()

@tool
def query_destinations(city: str) -> str:
    """Query destination information from database."""
    # Safe SQL query (no user input directly in SQL)
    cursor = db.cursor()
    cursor.execute(
        "SELECT city, country, avg_temp, best_season FROM destinations WHERE city LIKE ?", 
        (f"%{city}%",)
    )
    
    result = cursor.fetchone()
    if result:
        city, country, temp, season = result
        return f"🏙️ {city}, {country}: Avg temp {temp}°C, best time: {season}"
    else:
        return f"No destination data found for {city}"

@tool 
def get_all_destinations() -> str:
    """Get all available destinations from database."""
    cursor = db.cursor()
    cursor.execute("SELECT city, country FROM destinations")
    results = cursor.fetchall()
    
    if results:
        destinations = [f"{city}, {country}" for city, country in results]
        return "Available destinations: " + ", ".join(destinations)
    else:
        return "No destinations in database"

# Test database tools
print("🗄️ Testing database queries:")
print(get_all_destinations.invoke({}))
print(query_destinations.invoke({"city": "Tokyo"}))


## 4.4 MCP

This example spins up an in-process FastMCP server, registers a simple `weather_mcp` tool, and calls it via an MCP client — the same pattern used in `backend/main.py`. MCP spans will be captured automatically.


In [ ]:
from mcp.server import FastMCP
from mcp.shared.memory import create_connected_server_and_client_session
import asyncio

# Deterministic weather data
WEATHER_DB = {
    "prague": {"temperature": "6-14°C", "conditions": "Crisp mornings, light rain", "advice": "Layer up and carry a compact umbrella"},
    "bangkok": {"temperature": "27-33°C", "conditions": "Humid afternoons, evening storms", "advice": "Light fabrics, hydrate, bring a poncho"},
    "dubai": {"temperature": "24-36°C", "conditions": "Dry heat with breezy nights", "advice": "High-SPF sunscreen and breathable layers"},
    "barcelona": {"temperature": "14-24°C", "conditions": "Sunny with a coastal breeze", "advice": "Light jacket for evenings, sunscreen by day"},
    "tokyo": {"temperature": "10-22°C", "conditions": "Cool mornings, clear afternoons", "advice": "Layered outfits and comfortable rainproof shoes"},
}

def build_summary(destination: str) -> str:
    key = destination.split(",")[0].strip().lower()
    payload = WEATHER_DB.get(key)
    if payload:
        return (
            f"MCP Weather • {destination}: {payload['temperature']} with {payload['conditions']}. "
            f"Tip: {payload['advice']}."
        )
    return (
        f"MCP Weather • {destination}: Seasonal averages unavailable. "
        "Check a forecast a few days ahead and pack adaptable layers."
    )

# Start an in-memory server and call the tool via a client session
server = FastMCP(name="Weather MCP Demo", instructions="Deterministic weather tool for instrumentation demos.")

@server.tool(name="weather_mcp", description="Return a simple weather briefing for a destination.")
def weather_tool(destination: str) -> str:
    return build_summary(destination)

async def run_demo():
    async with create_connected_server_and_client_session(server._mcp_server, raise_exceptions=True) as session:
        await session.list_tools()  # populate tool metadata
        result = await session.call_tool("weather_mcp", {"destination": "Tokyo, Japan"})
        if result.isError:
            raise RuntimeError("MCP weather tool returned an error result")
        parts = []
        for block in result.content:
            text_val = getattr(block, "text", None)
            if text_val:
                parts.append(text_val)
        print("\n🌦️ MCP result:")
        print("\n".join(parts).strip())

print("🔌 MCP demo: running in-process FastMCP")
try:
    # If an event loop is already running (e.g., Jupyter), use top-level await
    loop = asyncio.get_running_loop()
    await run_demo()
except RuntimeError:
    # No running loop; create one
    asyncio.run(run_demo())


## Summary

### Key Takeaways:
- **Tools make agents practical** by connecting them to real systems
- **Safety first**: Always validate and sanitize inputs  
- **Error handling**: Gracefully handle API failures and database errors
- **MCP enables scalability** with distributed, observable tool architectures

### Next Steps:
1. Try building your own tools for specific use cases
2. Explore the full main.py implementation
3. Enable MCP and see the weather service in action
4. Move to Section 5: RAG + Agentic RAG

Great work! 🚀
